<h1>George W Bush VGG16 Convnet 3-7-2016</h1>

<strong>Abstract</strong> 
Trained VGG16 Convnet on 529 George W Bush images vs 1264 random images (excluding George W Bush) over 100 epochs resulting in 84% accuracy, test score of 1.22, 72.4% precision and 77.7% recall on our test set. Demonstrate it's possible to train a CNN to recognize a certain person's face among many other people's faces.

<strong>Details</strong> Implemented VGG16 and trained it to identify George W Bush. He was chosen because of his high number of images, we would like to see how a CNN with a lot of images of a single person performs. George W Bush has 529 images and there are about 5,748 other people with a total of 12,643 images. But, we will discard 90% of the other images for performance leaving 1264 images.

The final epoch of training on the CNN reaches 100% accuracy on test set and around 85% on validation set and took about 2 hours running on GeForce GT 650M 1GB. Images were resized to 100x100 dimensions. Models and weights have been persisted in the models folder.

<strong>Recommendations</strong>
<ul>
<li>Consider training with minimal number of faces from the person of interest and compare against much more other random faces.</li>
<li>Consider categorizing on many faces as a multi cateogorical problem rather than a binary problem.</li>
<li>Reduce overfitting</li>
</ul>

In [1]:
%load_ext autoreload

In [20]:
%autoreload 2
%matplotlib inline

import os
import fnmatch
import numpy as np
from matplotlib.pyplot import imshow 
from PIL import Image

from skimage import io
from skimage.transform import resize
import sklearn.metrics as metrics

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

In [3]:
data_path = '../data/'
data_lfw_path = data_path + 'lfw_cropped/'

batch_size = 32
nb_epoch = 100
img_rows, img_cols = 100, 100
train_size_percent = .85
validation_split = .15
random_discard_percent = .9

<h2>Preparing Data</h2>

In [4]:
def get_filenames_separated_from_target(target):
    files = []
    target_files = []
    
    for root, dirnames, filenames in os.walk(data_lfw_path):
        for dirname in dirnames:
                for filename in os.listdir(os.path.join(data_lfw_path, dirname)):
                    if filename.endswith(".jpg"):
                        f = os.path.join(root + dirname, filename)
                        if dirname == target:
                            target_files.append(f)
                        else:
                            files.append(f)
                            
    data_to_keep = int((1 - random_discard_percent) * len(files))
    np.random.shuffle(files)
    
    return target_files, files[:data_to_keep]

In [5]:
def get_train_and_test_sets(target_data, data):

    all_data = [(t, 1) for t in target_data] + [(t, 0) for t in data]
    np.random.shuffle(all_data)
    
    train_size = int(train_size_percent * len(all_data))
    X_train = np.array([x[0] for x in all_data[:train_size]])
    y_train = np.array([x[1] for x in all_data[:train_size]])
    X_test = np.array([x[0] for x in all_data[train_size:]])  
    y_test = np.array([x[1] for x in all_data[train_size:]])
    
    return (X_train, y_train), (X_test, y_test)

In [6]:
def image_read(f):
    return resize(io.imread(f), (img_rows, img_cols))

In [7]:
target_files, files = get_filenames_separated_from_target('George_W_Bush')

In [8]:
images = [image_read(f) for f in files]
target_images = [image_read(f) for f in target_files]

In [9]:
(X_train, y_train), (X_test, y_test) = get_train_and_test_sets(target_images, images)

In [10]:
X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)

In [11]:
Y_train = np_utils.to_categorical(y_train,2)
Y_test = np_utils.to_categorical(y_test,2)

In [12]:
len(target_files), len(files)

(529, 1264)

<h2>Training and Testing the CNN</h2>

Implementation of VGG-like convnet http://keras.io/examples/

In [ ]:
def VGG_16(optimizer, batch_size=16):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,img_rows,img_cols)))
    model.add(Convolution2D(32, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(32, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)
    
    return model

In [ ]:
model = VGG_16('sgd', batch_size)
early_stopping = EarlyStopping(monitor='loss', patience=10, mode='min')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=validation_split)

In [ ]:
json_string = model.to_json()
open('models/GeorgeWBushVGG16Convnet.json', 'w').write(json_string)
model.save_weights('models/GeorgeWBushVGG16Convnet.h5')

In [12]:
model = model_from_json(open('models/GeorgeWBushVGG16Convnet.json').read())
model.load_weights('models/GeorgeWBushVGG16Convnet.h5')

In [15]:
score = model.evaluate(X_test, Y_test, batch_size=16, show_accuracy=True, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

269/269 [==============================] - 3s     
('Test score:', 1.2175375433659466)
('Test accuracy:', 0.84386617100371752)


In [18]:
y_pred = model.predict_classes(X_test, batch_size=8)

269/269 [==============================] - 3s     


In [21]:
metrics.confusion_matrix(y_test, y_pred)

array([[164,  24],
       [ 18,  63]])

In [22]:
metrics.precision_score(y_test, y_pred)

0.72413793103448276

In [23]:
metrics.recall_score(y_test, y_pred)

0.77777777777777779